### **Random search experiment**

### Installing correct libraries


In [ ]:
!pip install auto-sklearn # Downgrade scipy to 1.4.x
!pip install scipy # Upgrade scipy to 1.7.x

# Restart runtime

     |████████████████████████████████| 6.3 MB 25.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.4 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
     |████████████████████████████████| 722 kB 37.2 MB/s 
     |████████████████████████████████| 4.2 MB 31.2 MB/s 
     |████████████████████████████████| 4.0 MB 34.6 MB/s 
     |████████████████████████████████| 208 kB 56.5 MB/s 
     |████████████████████████████████| 973 kB 45.1 MB/s 
     |████████████████████████████████| 132 kB 70.2 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.2-py3-none-any.whl size=6586738 sha256=c6c63decbb3a0ec7b823d0b048a32e3c5376cb1bba7c020b248c0b5ea92ef9c0
  Stored in directory: /root/.cache/pip/wheels/b6/ba/aa/ce7b8deb41019b9c1c6cd0b511d553350b7075ba1d3f0f545c
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7044 sha256=5aa04dcbd2ad9c152b0bf4004dbca8cffd1d6321a187c9be63e152ae6151fd

### Importing Libraries

In [ ]:
# Includes
# Dependencies
# Common
import os
import sys
import time

import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# AML
import autosklearn
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import sklearn.datasets
import sklearn.metrics
import autosklearn.classification

IncorrectPackageVersionError: ignored

### Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Load Datasets

In [ ]:
# Load datasets
import numpy as np
import sys
datasets_id = [41165, 41161, 41159, 41163,41142, 1468, 41164, 40996, 1111, 12, 41166, 41138, 41143, 1486, 41147, 41167, 41168, 1596, 41150, 40668,3, 23512, 41169, 1067,23517, 31, 41146, 40984, 54, 1461, 40981, 1590, 
4135, 40685, 1169, 40975, 41027, 1489, 1464] # Appendix D, Table 20 from the autosklearn2.0 paper



small_datasets_id = [] # For testing with a smaller dataset
small_datasets_id = [41159, 41163,41142, 1468, 41164, 40996, 1111, 12, 41166, 41138, 41143, 1486, 41167, 41168, 1596, 41150, 40668,3, 23512, 41169, 1067,23517, 31, 41146, 40984, 54, 1461, 40981, 1590, 
4135, 40685, 1169, 40975, 41027, 1489, 1464]
# small_datasets_id.extend([datasets_id[2], datasets_id[3]])
# small_datasets_id.append(41027)
print(small_datasets_id)

[41159, 41163, 41142, 1468, 41164, 40996, 1111, 12, 41166, 41138, 41143, 1486, 41167, 41168, 1596, 41150, 40668, 3, 23512, 41169, 1067, 23517, 31, 41146, 40984, 54, 1461, 40981, 1590, 4135, 40685, 1169, 40975, 41027, 1489, 1464]


### Running experiments

In [ ]:
# import sklearn.datasets
datasets = small_datasets_id
print(datasets)
## add the different search spaces

##
nRandomSearch = 5

for data_id in datasets:
  x, y = sklearn.datasets.fetch_openml(
      data_id=data_id,
      return_X_y=True,
      as_frame=True) # Return dataframe
  x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=1
    )
  # print(f"x_train: {x_train}")
  # print(f"[{data_id}] succesful")
  # continue
  # Set up classifier
  average_accuracy = 0
  accuracies = []
  randSeed = np.random.randint(0, 9999999)
  for i in range(nRandomSearch):
    print(f"Iteration: {i}")
    classifier = np.random.choice(["random_forest","adaboost","bernoulli_nb","decision_tree","extra_trees","gradient_boosting","k_nearest_neighbors","lda:shrinkage","liblinear_svc","libsvm_svc","mlp","multinomial_nb","passive_aggressive","qda","random_forest","sgd"])
    ensembleSize = np.random.randint(10, 60)
    ensembleNBest = ensembleSize
    maxModelsOnDisc = np.random.randint(1, ensembleSize)
    resamplingStrategy = np.random.choice(["holdout", "holdout-iterative-fit", "cv", "cv-iterative-fit"])

    cls = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=30,
        per_run_time_limit=10,
        memory_limit=None,
        seed=randSeed,
        ensemble_size = ensembleSize,
        ensemble_nbest = ensembleNBest,
        max_models_on_disc = maxModelsOnDisc,
        resampling_strategy = resamplingStrategy,
        include = {
            'classifier': ["random_forest"]
        },
        exclude = None
    )

    # cls = autosklearn.classification.AutoSklearnClassifier(
      # time_left_for_this_task=60,
      # per_run_time_limit=10,
      # seed=randSeed,
    #   memory_limit=None,
      # ensemble_size = ensembleSize,
      # ensemble_nbest = ensembleNBest,
      # max_models_on_disc = maxModelsOnDisc,
      # resampling_strategy = resamplingStrategy,
    #   n_jobs = -1,
    #   include = {
    #       'classifier': [classifier],
    #       'feature_preprocessor': ["no_preprocessing"]
    #   },
    #   exclude=None
    # )

    # Train
    try:
      cls.fit(x_train, y_train, x_test, y_test)

    # Predict
      predictions = cls.predict(x_test)
      accuracy = sklearn.metrics.accuracy_score(y_test, predictions)
      accuracies.append(accuracy)
      average_accuracy = average_accuracy + accuracy
      print(f"Iteration {i}, Dataset {data_id} with result {accuracy}")
    except:
      with open('/content/gdrive/MyDrive/AML_experiments/experiments.txt', 'a') as f:
        f.write(f'FAILED data set {data_id}')
      break
  else: # Only gets executed if the inner for loop does NOT break http://psung.blogspot.com/2007/12/for-else-in-python.html
    average_accuracy = average_accuracy/nRandomSearch
    print(f'For dataset [{data_id}] the average accuracy over {nRandomSearch} random search is {average_accuracy}. The seed used is {randSeed}')
    with open('/content/gdrive/MyDrive/AML_experiments/experiments.txt', 'a') as f:
      f.write(f'For dataset [{data_id}] the best accuracy is [{max(accuracies):.5f}]. The average accuracy over [{nRandomSearch}] random search is [{average_accuracy:.5f}]. The seed used is [{randSeed}]\n')





[41159, 41163, 41142, 1468, 41164, 40996, 1111, 12, 41166, 41138, 41143, 1486, 41167, 41168, 1596, 41150, 40668, 3, 23512, 41169, 1067, 23517, 31, 41146, 40984, 54, 1461, 40981, 1590, 4135, 40685, 1169, 40975, 41027, 1489, 1464]
Iteration: 0
[WARNING] [2021-12-02 23:49:51,409:Client-AutoML(2563386):81cbf594-53ca-11ec-8330-0242ac1c0002] Capping the per_run_time_limit to 9.0 to have time for a least 2 models in each process.
[WARNING] [2021-12-02 23:49:56,950:Client-AutoMLSMBO(2563386)::81cbf594-53ca-11ec-8330-0242ac1c0002] Configuration 546 not found
[WARNING] [2021-12-02 23:49:56,950:Client-AutoMLSMBO(2563386)::81cbf594-53ca-11ec-8330-0242ac1c0002] Configuration 690 not found
[WARNING] [2021-12-02 23:49:56,950:Client-AutoMLSMBO(2563386)::81cbf594-53ca-11ec-8330-0242ac1c0002] Configuration 81 not found
[WARNING] [2021-12-02 23:49:56,950:Client-AutoMLSMBO(2563386)::81cbf594-53ca-11ec-8330-0242ac1c0002] Configuration 32 not found
[WARNING] [2021-12-02 23:49:56,950:Client-AutoMLSMBO(256338

/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


[WARNING] [2021-12-03 00:01:34,652:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 375 not found
[WARNING] [2021-12-03 00:01:34,652:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 58 not found
[WARNING] [2021-12-03 00:01:34,653:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 538 not found
[WARNING] [2021-12-03 00:01:34,653:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 302 not found
[WARNING] [2021-12-03 00:01:34,653:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 40 not found
[WARNING] [2021-12-03 00:01:34,653:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 150 not found
[WARNING] [2021-12-03 00:01:34,653:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002] Configuration 232 not found
[WARNING] [2021-12-03 00:01:34,653:Client-AutoMLSMBO(9549781)::2765088a-53cc-11ec-8330-0242ac1c0002

/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


[WARNING] [2021-12-03 00:02:01,283:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 375 not found
[WARNING] [2021-12-03 00:02:01,283:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 58 not found
[WARNING] [2021-12-03 00:02:01,284:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 538 not found
[WARNING] [2021-12-03 00:02:01,284:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 302 not found
[WARNING] [2021-12-03 00:02:01,284:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 40 not found
[WARNING] [2021-12-03 00:02:01,284:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 150 not found
[WARNING] [2021-12-03 00:02:01,284:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002] Configuration 232 not found
[WARNING] [2021-12-03 00:02:01,284:Client-AutoMLSMBO(9549781)::3716f0da-53cc-11ec-8330-0242ac1c0002

...

[WARNING] [2021-12-03 00:02:53,081:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 375 not found
[WARNING] [2021-12-03 00:02:53,082:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 58 not found
[WARNING] [2021-12-03 00:02:53,083:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 538 not found
[WARNING] [2021-12-03 00:02:53,089:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 232 not found
[WARNING] [2021-12-03 00:02:53,083:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 302 not found
[WARNING] [2021-12-03 00:02:53,083:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 40 not found
[WARNING] [2021-12-03 00:02:53,089:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002] Configuration 150 not found
[WARNING] [2021-12-03 00:02:53,089:Client-AutoMLSMBO(9549781)::566dc0e6-53cc-11ec-8330-0242ac1c0002

/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


[WARNING] [2021-12-03 00:03:18,854:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 375 not found
[WARNING] [2021-12-03 00:03:18,854:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 58 not found
[WARNING] [2021-12-03 00:03:18,855:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 538 not found
[WARNING] [2021-12-03 00:03:18,855:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 302 not found
[WARNING] [2021-12-03 00:03:18,855:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 40 not found
[WARNING] [2021-12-03 00:03:18,855:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 150 not found
[WARNING] [2021-12-03 00:03:18,855:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002] Configuration 232 not found
[WARNING] [2021-12-03 00:03:18,856:Client-AutoMLSMBO(9549781)::65de15c6-53cc-11ec-8330-0242ac1c0002

/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')


Streaminguitvoer ingekort tot de laatste 5000 regels.
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 88 not found
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 71 not found
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 254 not found
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 270 not found
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 386 not found
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 395 not found
[WARNING] [2021-12-03 00:51:16,324:Client-AutoMLSMBO(4771952)::1d3deef8-53d3-11ec-8330-0242ac1c0002] Configuration 108 not found
[WARNING] [2021-12-03 00:51:16,324:Client-Aut